# Battery Drop Detect

In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import sys
sys.path.append('.')
sys.path.append('..')
import pandas as pd
import datetime
from service_batteryDropDetect.schedule import inference_pod
from common.db import Database_Connection
from common.log import init_logging
from common.utils import gpu_ram_config
from service_batteryDropDetect.f45movement import F45Movement
from service_batteryDropDetect.f45_inference_main_online import f45_battDrop_detect


**Create k8s Job**

In [ ]:
db = Database_Connection()
table_name = 'inference_schedule_setting'
cameraID_df = pd.read_sql(f"select server_id, mid from {table_name}", db.engine)
today = datetime.datetime.now().strftime("%Y%m%d")
cam = cameraID_df.iloc[0]
gpu_id = cam['server_id']
camera_mid = cam['mid']
job_name = cam['mid'].replace('_', '-').lower() + '-' + today
print(f'mid: {camera_mid}, job_name: {job_name}')
#下面這行在local沒辦法測試, 沒有k8s環境
#inference_pod(gpu_id, mid, job_name) 

In [ ]:
cameraID_df[cameraID_df['mid']=='F45_5L10']

**Inference**

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['ALIYUN_COM_GPU_MEM_IDX'] = '0'
os.environ['ALIYUN_POD_GPU_MEMORY'] = '1683'
os.environ['PRESERVE_GPU_MEMORY'] = '763'

init_logging()
#gpu_ram_config()

In [ ]:

endtime = datetime.datetime.now()+ datetime.timedelta(minutes=15)
f45_battDrop_detect(camera_mid, endtime.hour, endtime.minute, is_online=False, stream_mode=F45Movement.STEAM_MODE_TEST_FILE)


In [ ]:
f45_battDrop_detect(camera_mid, endtime.hour, endtime.minute, is_online=False, stream_mode=F45Movement.STEAM_MODE_REPLAY)
